# This notebook contains EDA,visualization and sentiment analysis on Covid-19 tweets

Downloaded tweets with the hashtag #covid19


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Load all the required libraries

In [ ]:
import numpy             as np
import pandas            as pd
import matplotlib.pyplot as plt
import seaborn           as sns
import plotly.graph_objs as go
import plotly.express    as px 
import nltk
import re
import string

from scipy.stats import norm
from wordcloud   import WordCloud, STOPWORDS

import warnings
warnings.filterwarnings("ignore")
from wordcloud import WordCloud,STOPWORDS
stopwords = set(STOPWORDS)

from textblob import TextBlob
import re
from collections import Counter

from sklearn.metrics import classification_report,accuracy_score,confusion_matrix
from IPython.display import Markdown as md


In [ ]:
data=pd.read_csv("../input/covid19-tweets/covid19_tweets.csv")

**Let's look at the dataset we have :**

user_name - contains the user name of that person.

user_location - contains the user's loaction from where he/she has tweeted.

user_description - It contains the user description on Tweeter

user_created - It contains the user id created time and date.

user_followers - It conatins the followers of users

user_friends - It contains the user's friends on Tweeter

user_favourites - It conatins user's favourites on Tweeter.

user_verified - User is verified or not ( True / False )

date - Date of Tweet

text - Text of Tweet he/she has tweeted.

hashtags - how many hashtags his/her tweet have.

source - It contains the source of that.

is_retweet - Any retweets it have or not ( True / False ).

In [ ]:
data.head()

In [ ]:
#Dtypes of features
data.info()

In [ ]:
#Description of the dataset
data.describe()

In [ ]:
#Number of rows and columns in the dataset
print("There are {} rows and {} columns in the dataset.".format(data.shape[0],data.shape[1]))

## Visualization

### 1. Word cloud

In [ ]:
text = ",".join(review for review in data.text if 'COVID' not in review and 'https' not in review and 'Covid' not in review)
wordcloud = WordCloud(max_words=200, colormap='Set3',background_color="black").generate(text)
plt.figure(figsize=(15,10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.figure(1,figsize=(12, 12))
plt.title('Prevalent words in tweets',fontsize=19)
plt.show()

### 2. Number of Tweets by location(Top 10)

In [ ]:
plt.figure(figsize=(10,12))
sns.barplot(data["user_location"].value_counts().values[0:10],
            data["user_location"].value_counts().index[0:10]);
plt.title("Top 10 Countries with maximum tweets",fontsize=14)
plt.xlabel("Number of tweets",fontsize=14)
plt.ylabel("Country",fontsize=14)
plt.show()

### 3. Heatmap representation of missing values

In [ ]:
plt.figure(figsize=(17, 5))
sns.heatmap(data.isnull(), cbar=True, cmap='Paired_r')
plt.xlabel("Column_Name", size=14, weight="bold")
plt.title("Places of missing values in column",fontweight="bold",size=14)
plt.show()

### 4. Bar plot of unique values in each column

In [ ]:
def unique_values_funct(data_frame):
    unique_dataframe = pd.DataFrame()
    unique_dataframe['Features'] = data_frame.columns
    uniques = []
    for col in data_frame.columns:
        u = data_frame[col].nunique()
        uniques.append(u)
    unique_dataframe['Uniques'] = uniques
    return unique_dataframe

udf = unique_values_funct(data)

f, ax = plt.subplots(1,1, figsize=(10,5))
sns.barplot(x=udf['Features'], y=udf['Uniques'], alpha=0.8)
plt.title('Bar plot for unique values in each column', fontsize=14)
plt.ylabel('Unique values', fontsize=14)
plt.xlabel('Columns', fontsize=14)
plt.xticks(rotation=90)
plt.show()

### 5. Distribution of words in text column

In [ ]:
data["num of words in text"] = data["text"].apply(lambda x: len(x))
plt.figure(figsize=(10,7))
sns.kdeplot(data["num of words in text"])
plt.title("Distribution of words in text column")
plt.xlabel("Number of words")
plt.show()

### 6. Users with maximum tweets(Top 20)

In [ ]:
username_count = data['user_name'].value_counts().reset_index().rename(columns={
    'user_name':'tweet_count','index':'user_name'})

plt.figure(figsize=(8, 10))
sns.barplot(y='user_name',x='tweet_count',data=username_count.head(20))
y=username_count['tweet_count'].head(20)
for index, value in enumerate(y):
    plt.text(value, index, str(value),fontsize=12)
plt.title('Users with maximum tweets',weight='bold', size=13)
plt.ylabel('Username', size=12, weight='bold')
plt.xlabel('TweetCount', size=12, weight='bold')
plt.show()

### 7. Plot verified users account

In [ ]:
plt.figure(figsize=(5, 5))
sns.countplot(x ="user_verified",data=data, palette="Set2")
plt.title("Verified user accounts or not ?")
plt.xticks([False,True],['Unverified','Verified'])
plt.show()

### 8. Plot platform with maximum number of tweets

In [ ]:
plt.figure(figsize=(15,5))
src = data['source'].value_counts().sort_values(ascending=False)
source = src.head(10)
source.plot.bar(color=['red', 'green', 'blue', 'black','cyan','pink','purple','violet','yellow','orange'])
plt.title('Platform with maximum number of tweets',size=13)
plt.xlabel('User Platform',size=13)
plt.ylabel('Tweet Count',size=13)
plt.show()

#### 8.1> 5 Most Tweet Sources used in India

In [ ]:
pla = data['source'][data['user_location'] == 'India'].value_counts().sort_values(ascending=False)
explode = (0, 0.1, 0, 0,0.01) 
plt.figure(figsize=(8,8))
pla[0:5].plot(kind = 'pie', title = '5 Most Tweet Sources used in India', autopct='%1.1f%%',shadow=True,explode = explode)
plt.show()

#### 8.2> 5 Most Tweet Sources used in USA

In [ ]:
pla = data['source'][data['user_location'] == 'United States'].value_counts().sort_values(ascending=False)
explode = (0, 0.1, 0, 0,0.01) 
plt.figure(figsize=(8,8))
pla[0:5].plot(kind = 'pie', title = '5 Most Tweet Sources used in USA', autopct='%1.1f%%',shadow=True,explode = explode)
plt.show()

#### 8.3> 5 Most Tweet Sources used in Switzerland

In [ ]:
pla = data['source'][data['user_location'] == 'Switzerland'].value_counts().sort_values(ascending=False)
explode = (0, 0.1, 0, 0,0.01) 
plt.figure(figsize=(8,8))
pla[0:5].plot(kind = 'pie', title = '5 Most Tweet Sources used in Switzerland', autopct='%1.1f%%',shadow=True,explode = explode)
plt.show()

#### 8.4> 5 Most Tweet Sources used in Australia

In [ ]:
pla = data['source'][data['user_location'] == 'Australia'].value_counts().sort_values(ascending=False)
explode = (0, 0.1, 0, 0,0.01) 
plt.figure(figsize=(8,8))
pla[0:5].plot(kind = 'pie', title = '5 Most Tweet Sources used in Australia', autopct='%1.1f%%',shadow=True,explode = explode)
plt.show()

#### 8.5> 5 Most Tweet Sources used in United Kingdom

In [ ]:
pla = data['source'][data['user_location'] == 'United Kingdom'].value_counts().sort_values(ascending=False)
explode = (0, 0.1, 0, 0,0.01) 
plt.figure(figsize=(8,8))
pla[0:5].plot(kind = 'pie', title = '5 Most Tweet Sources used in United Kingdom', autopct='%1.1f%%',shadow=True,explode = explode)
plt.show()

### 9. Plot top 5 hashtags

In [ ]:
top_tags=data['hashtags'].value_counts().sort_values(ascending=False)
plt.figure(figsize=(8,8))
explode = (0, 0.1, 0, 0,0.01) 

top_tags[0:5].plot(kind = 'pie',title = 'Top 5 hashtags',autopct='%1.1f%%',shadow=True,explode = explode)

### 10. Day with most number of tweets

In [ ]:
data["date"] = pd.to_datetime(data["date"])
data["Month"] = data["date"].apply(lambda x : x.month)
data["day"] = data["date"].apply(lambda x : x.dayofweek)
dmap = {0:'Mon',1:'Tue',2:'Wed',3:'Thu',4:'Fri',5:'Sat',6:'Sun'}
data["day"] = data["day"].map(dmap)
plt.title("Day with maximun tweets")
sns.countplot(data["day"])

## Sentiment Analysis on Covid19 Tweets

To peform sentiment analysis we need labeled dataset. The data can be downloaded from here: https://www.kaggle.com/surajkum1198/twitterdata

In [ ]:
senti_df = pd.read_csv('/kaggle/input/twitterdata/finalSentimentdata2.csv')

In [ ]:
senti_df

In [ ]:
senti_df.info()

In [ ]:
senti_df['sentiment'].nunique

In [ ]:
senti_df.describe()

In [ ]:
plt.figure(figsize=(10, 5))
sns.heatmap(senti_df.isnull(), cbar=True, cmap='Paired_r')
plt.xlabel("Column_Name", size=14, weight="bold")
plt.title("Places of missing values in column",fontweight="bold",size=14)
plt.show()

As we can see we dont have any missing/NaN value in our dataset.

## Data Preprocessing


Now let us preprocess text using some NLP tchniques like:

1. converting to lowercase
2. remove text in square brackets,
3. remove links,
4. remove punctuation
5. remove words containing numbers
6. Removing Punctuation
7. Removing stopwords
8. Stemming
9. Lemmatization

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

punc=string.punctuation
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    
    #Removing stopwords
    text=" ".join([word for word in str(text).split() if word not in stop_words])
    
    #Stemming
    text = " ".join([stemmer.stem(word) for word in text.split()])
    
    #Lemmatization
    text = " ".join([lemmatizer.lemmatize(word) for word in text.split()])
    
    return text

senti_df['text'] = senti_df['text'].apply(lambda x: clean_text(x))

Removing emojis

In [ ]:
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

senti_df['text']=senti_df['text'].apply(lambda x: remove_emoji(x))

In [ ]:
senti_df.head()

### Correct spelling of incorrect words

If you are using high configuration machine then only perform this step of correcting incorrect words.

In [ ]:
"""
!pip install textblob

from textblob import TextBlob

def correct_bytextblob(sent):
    return str(TextBlob(sent).correct())

senti_df['text'] = senti_df['text'].apply(lambda x: correct_bytextblob(x))

senti_df.to_csv('clean_tweets.csv',index=False)
"""

## Splitting the dataset

In [ ]:
from sklearn.model_selection import train_test_split

train,valid = train_test_split(senti_df,test_size = 0.2,random_state=0,stratify = senti_df.sentiment.values) #stratification means that the train_test_split method returns training and test subsets that have the same proportions of class labels as the input dataset.
print("train shape : ", train.shape)
print("valid shape : ", valid.shape)

## Vectorizing

I've checked vectorizing using TF-IDF technique but its not performing well so going ahead with CountVectorizer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
stop = list(stopwords.words('english'))
vectorizer = CountVectorizer(decode_error = 'replace',stop_words = stop)

X_train = vectorizer.fit_transform(train.text.values)
X_valid = vectorizer.transform(valid.text.values)

y_train = train.sentiment.values
y_valid = valid.sentiment.values

print("X_train.shape : ", X_train.shape)
print("X_train.shape : ", X_valid.shape)
print("y_train.shape : ", y_train.shape)
print("y_valid.shape : ", y_valid.shape)

Since the classes are almost balanced, we dont need to perform any sampling technique.

## ML model building

### Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB

naiveByes_clf = MultinomialNB()

naiveByes_clf.fit(X_train,y_train)

NB_prediction = naiveByes_clf.predict(X_valid)
NB_accuracy = accuracy_score(y_valid,NB_prediction)
print("training accuracy Score    : ",naiveByes_clf.score(X_train,y_train))
print("Validation accuracy Score : ",NB_accuracy )
print(classification_report(NB_prediction,y_valid))

### Stochastic Gradient Descent

In [ ]:
from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(loss = 'hinge', penalty = 'l2', random_state=0)

sgd_clf.fit(X_train,y_train)

sgd_prediction = sgd_clf.predict(X_valid)
sgd_accuracy = accuracy_score(y_valid,sgd_prediction)
print("Training accuracy Score    : ",sgd_clf.score(X_train,y_train))
print("Validation accuracy Score : ",sgd_accuracy )
print(classification_report(sgd_prediction,y_valid))

### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier()

rf_clf.fit(X_train,y_train)

rf_prediction = rf_clf.predict(X_valid)
rf_accuracy = accuracy_score(y_valid,rf_prediction)
print("Training accuracy Score    : ",rf_clf.score(X_train,y_train))
print("Validation accuracy Score : ",rf_accuracy )
print(classification_report(rf_prediction,y_valid))

#### Extreme Gradient Boosting

In [ ]:
#takes huge amount of time to execute
import xgboost as xgb

xgboost_clf = xgb.XGBClassifier()

xgboost_clf.fit(X_train, y_train)

xgb_prediction = xgboost_clf.predict(X_valid)
xgb_accuracy = accuracy_score(y_valid,xgb_prediction)
print("Training accuracy Score    : ",xgboost_clf.score(X_train,y_train))
print("Validation accuracy Score : ",xgb_accuracy )
print(classification_report(xgb_prediction,y_valid))

### Support Vector Machine 

In [ ]:
from sklearn.svm import SVC

svc = SVC()

svc.fit(X_train, y_train)

svc_prediction = svc.predict(X_valid)
svc_accuracy = accuracy_score(y_valid,svc_prediction)
print("Training accuracy Score    : ",svc.score(X_train,y_train))
print("Validation accuracy Score : ",svc_accuracy )
print(classification_report(svc_prediction,y_valid))

### Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()

logreg.fit(X_train, y_train)

logreg_prediction = logreg.predict(X_valid)
logreg_accuracy = accuracy_score(y_valid,logreg_prediction)
print("Training accuracy Score    : ",logreg.score(X_train,y_train))
print("Validation accuracy Score : ",logreg_accuracy )
print(classification_report(logreg_prediction,y_valid))

All the model test accuracy in descending order

In [ ]:
models = pd.DataFrame({
    'Model': ['Support Vector Machines', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes', 
              'Stochastic Gradient Decent', 'XGBoost'],
    'Test accuracy': [svc_accuracy, logreg_accuracy, 
              rf_accuracy, NB_accuracy, 
              sgd_accuracy, xgb_accuracy,]})

models.sort_values(by='Test accuracy', ascending=False)